In [3]:
# Import required libraries and dependencies
import pandas as pd
import hvplot.pandas
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

In [4]:
# Load the data into a Pandas DataFrame
df_market_data = pd.read_csv(
    "Resources/crypto_market_data.csv",
    index_col="coin_id")

# Display sample data
df_market_data.head(10)

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
coin_id,,,,,,,
bitcoin,1.08388,7.60278,6.57509,7.67258,-3.25185,83.51840,37.51761
ethereum,0.22392,10.38134,4.80849,0.13169,-12.88890,186.77418,101.96023
tether,-0.21173,0.04935,0.00640,-0.04237,0.28037,-0.00542,0.01954
ripple,-0.37819,-0.60926,2.24984,0.23455,-17.55245,39.53888,-16.60193
bitcoin-cash,2.90585,17.09717,14.75334,15.74903,-13.71793,21.66042,14.49384
binancecoin,2.10423,12.85511,6.80688,0.05865,36.33486,155.61937,69.69195
chainlink,-0.23935,20.69459,9.30098,-11.21747,-43.69522,403.22917,325.13186
cardano,0.00322,13.99302,5.55476,10.10553,-22.84776,264.51418,156.09756
litecoin,-0.06341,6.60221,7.28931,1.21662,-17.23960,27.49919,-12.66408


In [5]:
# Generate summary statistics
df_market_data.describe()

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
count,41.000000,41.000000,41.000000,41.000000,41.000000,41.000000,41.000000
mean,-0.269686,4.497147,0.185787,1.545693,-0.094119,236.537432,347.667956
std,2.694793,6.375218,8.376939,26.344218,47.365803,435.225304,1247.842884
min,-13.527860,-6.094560,-18.158900,-34.705480,-44.822480,-0.392100,-17.567530
25%,-0.608970,0.047260,-5.026620,-10.438470,-25.907990,21.660420,0.406170
50%,-0.063410,3.296410,0.109740,-0.042370,-7.544550,83.905200,69.691950
75%,0.612090,7.602780,5.510740,4.578130,0.657260,216.177610,168.372510
max,4.840330,20.694590,24.239190,140.795700,223.064370,2227.927820,7852.089700


In [6]:
# Plot your data to see what's in your DataFrame
df_market_data.hvplot.line(
    width=800,
    height=400,
    rot=90
)

:NdOverlay   [Variable]
   :Curve   [coin_id]   (value)

---

### Prepare the Data

In [7]:
# Use the `StandardScaler()` module from scikit-learn to normalize the data from the CSV file
scaler = StandardScaler()

In [8]:

# Create a DataFrame with the scaled data (no need to drop 'coin_id')
df_market_data_scaled = pd.DataFrame(
    scaler.fit_transform(df_market_data),
    columns=df_market_data.columns,
    index=df_market_data.index  # retain the original index
)

# Display the scaled DataFrame
print(df_market_data_scaled.head())


              price_change_percentage_24h  price_change_percentage_7d  \
coin_id                                                                 
bitcoin                          0.508529                    0.493193   
ethereum                         0.185446                    0.934445   
tether                           0.021774                   -0.706337   
ripple                          -0.040764                   -0.810928   
bitcoin-cash                     1.193036                    2.000959   

              price_change_percentage_14d  price_change_percentage_30d  \
coin_id                                                                  
bitcoin                          0.772200                     0.235460   
ethereum                         0.558692                    -0.054341   
tether                          -0.021680                    -0.061030   
ripple                           0.249458                    -0.050388   
bitcoin-cash                     1.760610   

---

### Find the Best Value for k Using the Original Scaled DataFrame.

In [9]:
# Create a list with the number of k-values from 1 to 11
k_values = list(range(1, 12))

In [10]:
# Create an empty list to store the inertia values
inertia_values = []

# Create a for loop to compute the inertia with each possible value of k
# Inside the loop:
# 1. Create a KMeans model using the loop counter for the n_clusters
# 2. Fit the model to the data using `df_market_data_scaled`
# 3. Append the model.inertia_ to the inertia list
for k in k_values:
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(df_market_data_scaled)
    inertia_values.append(kmeans.inertia_)


In [11]:
# Create a dictionary with the data to plot the Elbow curve
elbow_data = {"k": k_values, "inertia": inertia_values}

# Create a DataFrame with the data to plot the Elbow curve
elbow_df = pd.DataFrame(elbow_data)

In [12]:
# Plot a line chart with all the inertia values computed with
# the different values of k to visually identify the optimal value for k.
elbow_df.hvplot.line(
    x="k",
    y="inertia",
    title="Elbow Curve for Optimal k",
    width=800,
    height=400
)


:Curve   [k]   (inertia)

#### Answer the following question: 

**Question:** What is the best value for `k`?

**Answer:** The best value for "k" according this graph would be 4

---

### Cluster Cryptocurrencies with K-means Using the Original Scaled DataFrame

In [13]:
# Initialize the K-Means model using the best value for k
best_k = 4  # You can replace 4 with your identified optimal k from the graph


In [14]:
# Fit the K-Means model using the scaled DataFrame
kmeans = KMeans(n_clusters=best_k, random_state=42)
kmeans.fit(df_market_data_scaled)

KMeans(n_clusters=4, random_state=42)

In [15]:
# Predict the clusters to group the cryptocurrencies
cluster_labels = kmeans.predict(df_market_data_scaled)

# Print the resulting array of cluster values.
print(cluster_labels)

[0 0 2 2 0 0 0 0 0 2 2 2 2 0 2 0 2 2 0 2 2 0 2 2 2 2 2 2 0 2 2 2 3 0 2 2 1
 2 2 2 2]


In [16]:
# Create a copy of the scaled DataFrame
df_market_data_scaled_copy = df_market_data_scaled.copy()


In [17]:
# Add a new column to the copy of the scaled DataFrame with the predicted clusters
df_market_data_scaled_copy['Cluster'] = cluster_labels


# Display the copy of the scaled DataFrame
print(df_market_data_scaled_copy.head())


              price_change_percentage_24h  price_change_percentage_7d  \
coin_id                                                                 
bitcoin                          0.508529                    0.493193   
ethereum                         0.185446                    0.934445   
tether                           0.021774                   -0.706337   
ripple                          -0.040764                   -0.810928   
bitcoin-cash                     1.193036                    2.000959   

              price_change_percentage_14d  price_change_percentage_30d  \
coin_id                                                                  
bitcoin                          0.772200                     0.235460   
ethereum                         0.558692                    -0.054341   
tether                          -0.021680                    -0.061030   
ripple                           0.249458                    -0.050388   
bitcoin-cash                     1.760610   

In [18]:
# Create a scatter plot using hvPlot by setting
# `x="price_change_percentage_24h"` and `y="price_change_percentage_7d"`.
# Color the graph points with the labels found using K-Means and
# add the crypto name in the `hover_cols` parameter to identify
# the cryptocurrency represented by each data point.
df_market_data_scaled_copy.hvplot.scatter(
    x="price_change_percentage_24h",
    y="price_change_percentage_7d",
    c="Cluster",
    cmap="Set1",
    title="Cryptocurrency Clusters (Original Scaled Data)",
    hover_cols=["Cluster", "price_change_percentage_24h", "price_change_percentage_7d"],
    width=800,
    height=400
)


:Scatter   [price_change_percentage_24h]   (price_change_percentage_7d,Cluster)

---

### Optimize Clusters with Principal Component Analysis.

In [19]:
# Create a PCA model instance and set `n_components=3`.
pca = PCA(n_components=3)

In [20]:
# Use the PCA model with `fit_transform` to reduce the original scaled DataFrame
# down to three principal components.
df_market_data_pca = pca.fit_transform(df_market_data_scaled)

# View the scaled PCA data
print(df_market_data_pca[:5])

[[-0.60066733  0.84276006  0.46159457]
 [-0.45826071  0.45846566  0.95287678]
 [-0.43306981 -0.16812638 -0.64175193]
 [-0.47183495 -0.22266008 -0.47905316]
 [-1.15779997  2.04120919  1.85971527]]


In [21]:
# Retrieve the explained variance to determine how much information
# can be attributed to each principal component.
print(f"Explained variance by each PC: {pca.explained_variance_ratio_}")
print(f"Total explained variance: {pca.explained_variance_ratio_.sum()}")


Explained variance by each PC: [0.3719856  0.34700813 0.17603793]
Total explained variance: 0.895031657030984


#### Answer the following question: 

**Question:** What is the total explained variance of the three principal components?

**Answer:**  The total explained variance of the three principal components is 89.5%

In [22]:
# Create a new DataFrame with the PCA data.
df_market_data_pca_df = pd.DataFrame(df_market_data_pca, columns=["PC1", "PC2", "PC3"])

# Copy the crypto names from the original scaled DataFrame
df_market_data_pca_df['coin_id'] = df_market_data_scaled_copy.index

# Set the coin_id column as index
df_market_data_pca_df.set_index('coin_id', inplace=True)

# Display the scaled PCA DataFrame
print(df_market_data_pca_df.head())

                   PC1       PC2       PC3
coin_id                                   
bitcoin      -0.600667  0.842760  0.461595
ethereum     -0.458261  0.458466  0.952877
tether       -0.433070 -0.168126 -0.641752
ripple       -0.471835 -0.222660 -0.479053
bitcoin-cash -1.157800  2.041209  1.859715


---

### Find the Best Value for k Using the Scaled PCA DataFrame

In [23]:
# Create a list with the number of k-values from 1 to 11
k_values_pca = list(range(1, 12))


In [24]:
# Create an empty list to store the inertia values
inertia_values_pca = []

# Create a for loop to compute the inertia with each possible value of k
# Inside the loop:
# 1. Create a KMeans model using the loop counter for the n_clusters
# 2. Fit the model to the data using `df_market_data_pca`
# 3. Append the model.inertia_ to the inertia list
for k in k_values_pca:
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(df_market_data_pca_df)
    inertia_values_pca.append(kmeans.inertia_)

In [25]:
# Create a dictionary with the data to plot the Elbow curve
elbow_data_pca = {"k": k_values_pca, "inertia": inertia_values_pca}

# Create a DataFrame with the data to plot the Elbow curve
elbow_df_pca = pd.DataFrame(elbow_data_pca)


In [35]:
# Plot a line chart with all the inertia values computed with
# the different values of k to visually identify the optimal value for k.
elbow_df_pca.hvplot.line(
    x="k",
    y="inertia",
    title="Elbow Curve for Optimal k (PCA Data)",
    width=800,
    height=400
)

:Curve   [k]   (inertia)

#### Answer the following questions: 

* **Question:** What is the best value for `k` when using the PCA data?

  * **Answer:** The best value for "k" using the PCA data is 4.


* **Question:** Does it differ from the best k value found using the original data?

  * **Answer:** No the optimal "k" value for both is 4

### Cluster Cryptocurrencies with K-means Using the Scaled PCA DataFrame

In [37]:
# Initialize the K-Means model using the best value for k
best_k_pca = 4

In [39]:
# Fit the K-Means model using the PCA data
kmeans_pca = KMeans(n_clusters=best_k_pca, random_state=42)
kmeans_pca.fit(df_market_data_pca_df)

KMeans(n_clusters=4, random_state=42)

In [40]:
# Predict the clusters to group the cryptocurrencies using the scaled PCA DataFrame
cluster_labels_pca = kmeans_pca.predict(df_market_data_pca_df)

# Print the resulting array of cluster values.
print(cluster_labels_pca)


[0 0 2 2 0 0 0 0 0 2 2 2 2 0 2 0 2 2 0 2 2 0 2 2 2 2 2 2 0 2 2 2 3 0 2 2 1
 2 2 2 2]


In [41]:
# Create a copy of the scaled PCA DataFrame
df_market_data_pca_copy = df_market_data_pca_df.copy()

# Add a new column to the copy of the PCA DataFrame with the predicted clusters
df_market_data_pca_copy['Cluster'] = cluster_labels_pca

# Display the copy of the scaled PCA DataFrame
print(df_market_data_pca_copy.head())

                   PC1       PC2       PC3  Cluster
coin_id                                            
bitcoin      -0.600667  0.842760  0.461595        0
ethereum     -0.458261  0.458466  0.952877        0
tether       -0.433070 -0.168126 -0.641752        2
ripple       -0.471835 -0.222660 -0.479053        2
bitcoin-cash -1.157800  2.041209  1.859715        0


In [42]:
# Create a scatter plot using hvPlot by setting
# `x="PC1"` and `y="PC2"`.
# Color the graph points with the labels found using K-Means and
# add the crypto name in the `hover_cols` parameter to identify
# the cryptocurrency represented by each data point.
df_market_data_pca_copy.hvplot.scatter(
    x="PC1",
    y="PC2",
    c="Cluster",
    cmap="Set1",
    title="Cryptocurrency Clusters (PCA Data)",
    hover_cols=["Cluster", "PC1", "PC2"],
    width=800,
    height=400
)

:Scatter   [PC1]   (PC2,Cluster)

### Visualize and Compare the Results

In this section, you will visually analyze the cluster analysis results by contrasting the outcome with and without using the optimization techniques.

In [47]:
# Composite plot to contrast the Elbow curves
# Plot both Elbow curves on the same figure for comparison
elbow_original_plot = elbow_df.hvplot.line(
    x="k",
    y="inertia",
    title="Elbow Curves for Original vs PCA Data",
    line_color="blue",
    label="Original Data",
    width=800,
    height=400
)

elbow_pca_plot = elbow_df_pca.hvplot.line(
    x="k",
    y="inertia",
    line_color="green",
    label="PCA Data"
)

# Combine the two plots
elbow_original_plot * elbow_pca_plot


:Overlay
   .Curve.Original_Data :Curve   [k]   (inertia)
   .Curve.PCA_Data      :Curve   [k]   (inertia)

In [48]:
# Composite plot to contrast the clusters
# Add coin_id back as a column for hover tooltips
df_market_data_scaled_copy['coin_id'] = df_market_data_scaled_copy.index
df_market_data_pca_copy['coin_id'] = df_market_data_pca_copy.index

# Plot: Clusters from original scaled data
original_clusters_plot = df_market_data_scaled_copy.hvplot.scatter(
    x="price_change_percentage_24h",
    y="price_change_percentage_7d",
    c="Cluster",
    cmap="Set1",
    title="Clusters Using Original Scaled Data",
    hover_cols=["coin_id"],
    width=400,
    height=400
)

# Plot: Clusters from PCA data
pca_clusters_plot = df_market_data_pca_copy.hvplot.scatter(
    x="PC1",
    y="PC2",
    c="Cluster",
    cmap="Set1",
    title="Clusters Using PCA Data",
    hover_cols=["coin_id"],
    width=400,
    height=400
)

# Combine plots side-by-side
original_clusters_plot + pca_clusters_plot


:Layout
   .Scatter.I  :Scatter   [price_change_percentage_24h]   (price_change_percentage_7d,Cluster,coin_id)
   .Scatter.II :Scatter   [PC1]   (PC2,Cluster,coin_id)

#### Answer the following question: 

  * **Question:** After visually analyzing the cluster analysis results, what is the impact of using fewer features to cluster the data using K-Means?

  * **Answer:** Using fewer features (PCA) simplifies the data while preserving most variance. In the visual comparison, both clustering results (original and PCA) identified similar groupings, suggesting that PCA retained enough structure for effective clustering. However, the PCA-based plot shows tighter and more separated clusters, which can make the results easier to interpret and visualize. This suggests that dimensionality reduction not only reduces complexity but may also improve cluster clarity.